In [29]:
import pyspark
import findspark
findspark.init()
from pyspark import SparkContext

#import wget

#wget.download("https://s3.amazonaws.com/metcs777/taxi-data-sorted-small.csv.bz2")
sc = SparkContext.getOrCreate()  

def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False
    

def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) !=0 and float(p[11]) !=0):
                return p

#Getting the Data            
lines = sc.textFile("taxi-data-sorted-small.csv.bz2")

words = lines.map(lambda x: x.split(','))

#Data Cleaning
wordsCorr = words.filter(lambda x: correctRows(x))
    
wordsAstuples = words.map(lambda x: (x[0], [x[1], 1]))
    
medDriver = wordsAstuples.map(lambda x: (x[0], x[1][1]))
    
counts = medDriver.reduceByKey(lambda x, y: x+y)
    
top10 = counts.top(10, lambda x: x[1])

In [30]:
top10

[('6FFCF7A4F34BA44239636028E680E438', 585),
 ('D5C7CD37EA4D372D00F0A681CDC93F11', 569),
 ('849E486825860106403FB991A763BCC3', 558),
 ('A979CDA04CFB8BA3D3ACBA7E8D7F0661', 553),
 ('DA1A4CB0E75444C73D1D1633E701206E', 550),
 ('A532B1493C4DD88C450F6796369EAA6F', 545),
 ('818B2426C5493017D5CFE68EFD34617E', 539),
 ('FF40FB8123940D9F96D33EDA1D92A83C', 536),
 ('075E4BFE6607421289B566A32BC135E5', 535),
 ('1A50563CC49E045A115AE0BA61BE4658', 531)]

In [12]:
def zero_check(x):
    if (x == 0): 
        return 0.0001
    else:
        return x

In [13]:
drive_mon = words.map(lambda x: (x[1], (float(x[14])) + float(x[16])))
drive_time = words.map(lambda x: (x[1], float(x[4])/60))
mon_total = drive_mon.reduceByKey(lambda x, y: x+y)
time_total = drive_time.reduceByKey(lambda x, y: x+y)

In [19]:
mon_total.take(2)

[('778C92B26AE78A9EBDF96B49C67E4007', 1500.4600000000003),
 ('25BA06A87905667AA1FE5990E33F0E2E', 2922.26)]

In [22]:
mon_total_cor = mon_total.map(lambda x: (x[0], zero_check(x[1])))
time_total_cor = time_total.map(lambda x: (x[0], zero_check(x[1])))

In [23]:
mon_time = mon_total_cor.join(time_total_cor)
average = mon_time.map(lambda x: (x[0], x[1][0]/x[1][1]))

In [24]:
mon_total_cor.take(2)

[('778C92B26AE78A9EBDF96B49C67E4007', 1500.4600000000003),
 ('25BA06A87905667AA1FE5990E33F0E2E', 2922.26)]

In [25]:
mon_time.take(2)

[('778C92B26AE78A9EBDF96B49C67E4007', (1500.4600000000003, 1037.0)),
 ('25BA06A87905667AA1FE5990E33F0E2E', (2922.26, 2514.0))]

In [31]:
top10_average = average.top(10, lambda x: x[1])

In [28]:
average.take(3)

[('778C92B26AE78A9EBDF96B49C67E4007', 1.4469238187078113),
 ('25BA06A87905667AA1FE5990E33F0E2E', 1.1623945902943518),
 ('577D1B6729EF9014CDBC949554EB4483', 1.504027027027027)]

In [32]:
top10_average

[('F90DB0A8AD3F7A013C54E2DCD6A34AFA', 525000.0),
 ('73E45DAAA5E37264D19AF79422C33D34', 110000.0),
 ('7BD4876222813A92A8389D03EEE7B1FF', 60000.0),
 ('9C3CCC83B0486021F0A16A68951F53CA', 40000.0),
 ('6789302E98F439768EF7D0CC22666B92', 35000.0),
 ('0C7717E4EE9A0079B47E975417279255', 35000.0),
 ('689153C038F51E19A47257E15DF838BD', 35000.0),
 ('610F4C786A28108FEE10A5A5F5DBEE7F', 30000.0),
 ('E65E9C993E76354E84ABFE5295F121E2', 30000.0),
 ('1AAAB8B74F8E2D79C8C5B0BAB486704D', 30000.0)]